# Sequence Models - RNNs

We have developed some fairly sophisticated NLP models, but none has made use of the *order* of words, with the partial exception of word embeddings.

What might our models therefore be missing out on?

The idea of an RNN is to _remember_ recent inputs during training. So the network will make use not just of inputs and weights but also of _internal states_, which capture information about previous time-steps.

Recurrent models process data *in sequence*, so they are useful for analyzing:

- time-series data
- text
- drawings

Here's a demo of a drawing-to-'photo' network: [Image-to-Image](https://affinelayer.com/pixsrv/)

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from sklearn.feature_extraction.text import TfidfVectorizer

Using TensorFlow backend.


## Reading in the Twitter data

In [2]:
import pandas as pd
tweets = pd.read_csv('Tweets.csv')
tweets.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
tweets.shape

(14640, 15)

In [4]:
tweets['text'][0]

'@VirginAmerica What @dhepburn said.'

In [5]:
# We'll use the translate method to eliminate the punctuation
# from our strings.

import string
data = []
for tweet in tweets['text']:
    sentence = tweet.translate(str.maketrans('', '', string.punctuation))
    data.append(sentence)

In [6]:
data[0]

'VirginAmerica What dhepburn said'

In [7]:
# The TF-IDF Vectorizer will look for words uncommon in the corpus
# as a whole.

tfidf = TfidfVectorizer(stop_words = 'english')
tfidf.fit(data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [8]:
twitter_df = pd.DataFrame(tfidf.transform(data).todense(), columns=tfidf.get_feature_names())
twitter_df.head()

,00,0011,0016,006,0162389030167,0162424965446,0162431184663,0167560070877,0214,021mbps,...,zkatcher,zombie,zone,zones,zoom,zrh,zrhairport,zukes,zurich,zurichnew
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
twitter_df.shape

(14640, 16349)

In [10]:
tweets['airline_sentiment'].value_counts()

negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64

In [11]:
sent_dict = {'negative': 0, 'neutral': 1, 'positive': 2}
tweets['airline_sentiment'] = tweets['airline_sentiment'].map(sent_dict)

In [12]:
X = twitter_df
y = tweets['airline_sentiment']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                   random_state=42)

In [14]:
y_train_c = to_categorical(y_train)
y_test_c = to_categorical(y_test)

In [16]:
model = Sequential()
inputs = X_train.shape[1]

model.add(Dense(inputs, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(3, activation='softmax'))

In [18]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [19]:
history_log = model.fit(np.asarray(X_train), y_train_c,
                        validation_data=(np.asarray(X_test), y_test_c),
                        epochs=5,
                       batch_size=2000)

W0715 16:00:04.638894 4702528960 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0715 16:00:04.657161 4702528960 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0715 16:00:04.772367 4702528960 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0715 16:00:04.920970 4702528960 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has th

Train on 10980 samples, validate on 3660 samples
Epoch 1/5
10980/10980 [==============================] - 302s 28ms/step - loss: 0.9495 - acc: 0.5522 - val_loss: 0.8299 - val_acc: 0.6268
Epoch 2/5
10980/10980 [==============================] - 353s 32ms/step - loss: 0.7096 - acc: 0.6286 - val_loss: 0.6931 - val_acc: 0.6459
Epoch 3/5
10980/10980 [==============================] - 371s 34ms/step - loss: 0.5381 - acc: 0.7117 - val_loss: 0.6309 - val_acc: 0.7153
Epoch 4/5
10980/10980 [==============================] - 380s 35ms/step - loss: 0.3954 - acc: 0.8542 - val_loss: 0.6138 - val_acc: 0.7544
Epoch 5/5
10980/10980 [==============================] - 865s 79ms/step - loss: 0.2852 - acc: 0.9338 - val_loss: 0.6463 - val_acc: 0.7697


## This time with an LSTM

Long short-term memory network: Architecturally speaking, the networks have *forget* gates in addition to input and output gates. These forget gates control the extent to which previous states affect future states.

Because of this capacity for memory, LSTMs (and RNNs in general) have been compared to [Turing  machines](https://medium.com/@Lordunlocked/turing-machines-what-are-they-and-why-you-should-care-aaf030c37d40), and there is some debates about whether it makes sense to think of RNNs as Turing-complete.

In [176]:
data = []
for tweet in tweets['text'][:1000]:
    sentence = tweet.translate(str.maketrans('', '', string.punctuation))
    data.append(sentence)

In [177]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf.fit(data)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [178]:
twitter_df = pd.DataFrame(tfidf.transform(data).todense(), columns=tfidf.get_feature_names())
twitter_df.head()

,00,0011,0016,0162424965446,0530,0736,0769,10,100,1000,...,youve,yr,yrs,yulord,yummy,yvr,zambia,zcc82u,zero,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.390956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [179]:
X = twitter_df
y = tweets['airline_sentiment'][:1000]

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                   random_state=42)

In [181]:
y_train_c = to_categorical(y_train)
y_test_c = to_categorical(y_test)

In [182]:
X_train_r = np.reshape(np.asarray(X_train), (X_train.shape[0], 1, X_train.shape[1]))
X_test_r = np.reshape(np.asarray(X_test), (X_test.shape[0], 1, X_test.shape[1]))
y_train_r = np.reshape(np.asarray(y_train_c), (y_train_c.shape[0], 1, y_train_c.shape[1]))
y_test_r = np.reshape(np.asarray(y_test_c), (y_test_c.shape[0], 1, y_test_c.shape[1]))

In [183]:
rnn = Sequential()

inputs = X_train_r.shape[2]

rnn.add(LSTM(inputs, input_shape=(1, inputs), return_sequences=True))
rnn.add(LSTM(200, return_sequences=True))
rnn.add(LSTM(30, return_sequences=True))
rnn.add(Dense(3, activation='softmax'))

In [186]:
rnn.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [187]:
# Fitting our model

rnn.fit(X_train_r, y_train_r,
          validation_data = (X_test_r, y_test_r), epochs=5)

Train on 750 samples, validate on 250 samples
Epoch 1/5
750/750 [==============================] - 39s 52ms/step - loss: 0.0379 - acc: 0.9933 - val_loss: 2.1119 - val_acc: 0.5600
Epoch 2/5
750/750 [==============================] - 31s 41ms/step - loss: 0.0190 - acc: 0.9973 - val_loss: 2.1131 - val_acc: 0.5840
Epoch 3/5
750/750 [==============================] - 30s 40ms/step - loss: 0.0178 - acc: 0.9960 - val_loss: 2.3918 - val_acc: 0.5200
Epoch 4/5
750/750 [==============================] - 32s 42ms/step - loss: 0.0164 - acc: 0.9960 - val_loss: 2.2613 - val_acc: 0.5600
Epoch 5/5
750/750 [==============================] - 30s 40ms/step - loss: 0.0132 - acc: 0.9947 - val_loss: 2.1442 - val_acc: 0.5960
